In [13]:
import pandas as pd
import numpy as np
import json

from sklearn.preprocessing import MinMaxScaler

def data_loader(data_path, city, year, level='district', length=12, n_steps=12, is_scale=False, temporal_copy=False, is_realtime=False, train_ratio=0.8):
    
    def normalize(train, test):
        if is_scale:
            scaler = MinMaxScaler()
            train_shape, test_shape = train.shape, test.shape
            train = scaler.fit_transform(train.reshape(-1, train_shape[-1]))
            test = scaler.transform(test.reshape(-1, test_shape[-1]))
            return train.reshape(train_shape), test.reshape(test_shape), scaler
        else:
            return train, test, None

    risk_data = pd.read_csv(f'{data_path}/risk_scores/{city}-{year}-{level}-hour-risk.csv')
    selected_areas = risk_data.drop(columns=['date', 'time']).columns
    n_districts = len(selected_areas) # number of districts
    n_outputs = len(selected_areas)
    train_length = int(30 * train_ratio)

    risk_train, y_train = [], []
    risk_test, y_test = [], []
    for i in range(length, 721-n_steps):
        if i <= (train_length * 24): # before date 25th
            y_train.append(risk_data.drop(columns=['date', 'time']).iloc[i:i+n_steps, :n_outputs].to_numpy())
            risk_train.append(risk_data.drop(columns=['date', 'time']).iloc[i-length:i, :n_districts].to_numpy())
        else:
            y_test.append(risk_data.drop(columns=['date', 'time']).iloc[i:i+n_steps, :n_outputs].to_numpy())
            risk_test.append(risk_data.drop(columns=['date', 'time']).iloc[i-length:i, :n_districts].to_numpy())
        
    risk_train, risk_test, risk_scaler = normalize(np.array(risk_train), np.array(risk_test))
    y_train, y_test = np.array(y_train), np.array(y_test)
    y_train_scaled, y_test_scaled, y_scaler = normalize(y_train, y_test)

    # Weather & Air Quality  
    weather_data = pd.read_csv(f'{data_path}/weather/{city}-{year}-count.csv').fillna(0)
    if level == 'district':
        weather_data['location'] = weather_data['location'].apply(lambda x: x.split('|')[0])
        weather_data = weather_data.groupby(by=['date','time','location'], as_index=False).mean()                
    weather_train, weather_test = [], []

    location_weather = []
    for location in selected_areas:
        location_weather.append(weather_data[weather_data['location'] == location].iloc[:, 3:].to_numpy())

    location_weather = np.concatenate(location_weather, axis=1)

    for i in range(length, 721-n_steps):
        if i <= (train_length * 24): # before date 25th
            weather_train.append(location_weather[i-length:i])
        else:
            weather_test.append(location_weather[i-length:i])
    
    weather_train, weather_test, _ = normalize(np.array(weather_train).reshape(len(weather_train), length, n_districts, -1), np.array(weather_test).reshape(len(weather_test), length, n_districts, -1))


    # Dangerous Driving Behavior
    dtg_data = pd.read_csv(f'{data_path}/dangerous_cases/{city}-{year}-date-hour-{level}-new.csv')
    dtg_train, dtg_test = [], []

    location_dtg = []
    for location in selected_areas:
        if level == 'district':
            district = location.split('|')[0]
            location_dtg.append(dtg_data[dtg_data['district'] == district].iloc[:, 3:].to_numpy())
        else:
            district, subdistrict = location.split('|')[0], location.split('|')[1]
            location_dtg.append(dtg_data[(dtg_data['district'] == district) & (dtg_data['subdistrict'] == subdistrict)].iloc[:, 3:].to_numpy())

    location_dtg = np.concatenate(location_dtg, axis=1)

    for i in range(length, 721-n_steps):
        if i <= (train_length * 24): # before date 25th
            dtg_train.append(location_dtg[i-length:i])
        else:
            dtg_test.append(location_dtg[i-length:i])

    dtg_train, dtg_test, _ = normalize(np.array(dtg_train).reshape(len(dtg_train), length, n_districts, -1), np.array(dtg_test).reshape(len(dtg_test), length, n_districts, -1))


    # Road data
    road_data = pd.read_csv(f'{data_path}/roads/{city}-{year}-{level}-road-count.csv').drop(columns=['attribute'])
    road_train, road_test = [], []

    location_road = []
    for location in selected_areas:
        location_road.append(road_data[location].to_numpy())

    for i in range(length, 721-n_steps):
        if i <= (train_length * 24): # before date 25th
            road_train.append(np.array([location_road]*length)) if temporal_copy else road_train.append(np.array(location_road))
        else:
            road_test.append(np.array([location_road]*length)) if temporal_copy else road_test.append(np.array(location_road))
            
    road_train, road_test, _ = normalize(np.array(road_train), np.array(road_test))


    # demographics data
    demo_data = pd.read_csv(f'{data_path}/demographic/{city}-{year}-{level}.csv').drop(columns=['index'])
    demo_train, demo_test = [], []

    location_demo = []
    for location in selected_areas:
        location_demo.append(demo_data[location].to_numpy())

    for i in range(length, 721-n_steps):
        if i <= (train_length * 24): # before date 25th
            demo_train.append(np.array([location_demo]*length)) if temporal_copy else demo_train.append(np.array(location_demo))
        else:
            demo_test.append(np.array([location_demo]*length)) if temporal_copy else demo_test.append(np.array(location_demo))
            
    demo_train, demo_test, _ = normalize(np.array(demo_train), np.array(demo_test))


    # POI data
    poi_data = pd.read_csv(f'{data_path}/poi/{city}-{year}-{level}.csv').drop(columns=['location'])
    poi_train, poi_test = [], []

    location_poi = []
    for location in selected_areas:
        location_poi.append(poi_data[location].to_numpy())

    for i in range(length, 721-n_steps):
        if i <= (train_length * 24): # before date 25th
            poi_train.append(np.array([location_poi]*length)) if temporal_copy else poi_train.append(np.array(location_poi))
        else:
            poi_test.append(np.array([location_poi]*length)) if temporal_copy else poi_test.append(np.array(location_poi))
            
    poi_train, poi_test, _ = normalize(np.array(poi_train), np.array(poi_test))


    # traffic volumes
    volume_data = pd.read_csv(f'{data_path}/traffic_volume/{city}-{year}.csv').drop(columns=['date', 'hour'])
    volume_train, volume_test = [], []

    for i in range(length, 721-n_steps):
        if i <= (train_length * 24): # before date 25th
            volume_train.append(volume_data.iloc[i-length:i, :n_districts].to_numpy())
        else:
            volume_test.append(volume_data.iloc[i-length:i, :n_districts].to_numpy())

    volume_train, volume_test, _ = normalize(np.array(volume_train), np.array(volume_test))
    

    # traffic speed
    speed_data = pd.read_csv(f'{data_path}/traffic_speed/{city}-{year}.csv').drop(columns=['date', 'hour'])
    speed_train, speed_test = [], []

    for i in range(length, 721-n_steps):
        if i <= (train_length * 24): # before date 25th
            speed_train.append(speed_data.iloc[i-length:i, :n_districts].to_numpy())
        else:
            speed_test.append(speed_data.iloc[i-length:i, :n_districts].to_numpy())

    speed_train, speed_test, _ = normalize(np.array(speed_train), np.array(speed_test))
    

    # calendar
    calendar_data = pd.read_csv(f'{data_path}/calendar/calendar-{city}-{year}-{level}.csv')
    calendar_train, calendar_test = [], []
    
    location_calendar = []
    for location in selected_areas:
        location_calendar.append(calendar_data[calendar_data['location'] == location].iloc[:, 1:].to_numpy())

    location_calendar = np.concatenate(location_calendar, axis=1)

    for i in range(length, 721-n_steps):
        if i <= (train_length * 24): # before date 25th
            calendar_train.append(location_calendar[i:i+n_steps]) if is_realtime else calendar_train.append(location_calendar[i-length:i])
        else:
            calendar_test.append(location_calendar[i:i+n_steps]) if is_realtime else calendar_test.append(location_calendar[i-length:i])
    calendar_train, calendar_test = np.array(calendar_train), np.array(calendar_test)        
    calendar_train, calendar_test, _ = normalize(calendar_train.reshape(calendar_train.shape[0], calendar_train.shape[1], n_districts, -1), calendar_test.reshape(calendar_test.shape[0], calendar_test.shape[1], n_districts, -1))
    
    # Match Shape
    risk_train = risk_train[:,:,:,None]
    risk_test = risk_test[:,:,:,None]
    volume_train = volume_train[:,:,:,None]
    volume_test = volume_test[:,:,:,None]
    speed_train = speed_train[:,:,:,None]
    speed_test = speed_test[:,:,:,None]

    return {
        'risk': [risk_train, risk_test],
        'road': [road_train, road_test],
        'poi': [poi_train, poi_test],
        'demo': [demo_train, demo_test],
        'weather': [weather_train, weather_test],
        'calendar': [calendar_train, calendar_test],
        'volume': [volume_train, volume_test],
        'speed': [speed_train, speed_test],
        'dtg': [dtg_train, dtg_test],
        'y': [y_train, y_test],
        'y_scaled': [y_train_scaled, y_test_scaled],
        'selected_areas': selected_areas,
        'scaler': risk_scaler
    }

In [14]:
import numpy as np
import tensorflow as tf

def metric(pred, real):
    mae = np.mean(np.abs(pred - real))
    rmse = np.sqrt(np.mean(np.square(pred - real)))
    
    # PCC Calculation
    pred_flat = pred.flatten()
    real_flat = real.flatten()
    if len(pred_flat) > 0 and len(real_flat) > 0:
        pcc = np.corrcoef(pred_flat, real_flat)[0, 1]
    else:
        pcc = 0
    return mae, rmse, pcc

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, Model, backend as K

class STE_Layer(layers.Layer):
    """
    3. Lớp nhúng không gian - thời gian (STE Module)
    Kết hợp đặc trưng nút (Spatial) và nhúng thời gian (Temporal).
    """
    def __init__(self, output_dim, n_nodes, input_steps):
        super(STE_Layer, self).__init__()
        self.output_dim = output_dim
        self.n_nodes = n_nodes
        self.input_steps = input_steps
        # Temporal Embedding: Mã hóa thời gian (Time of day / Day of week)
        # Giả sử đầu vào time encoding nằm ở các feature cuối của input
        self.time_embedding = layers.Dense(output_dim, activation='relu') 
        self.feature_embedding = layers.Dense(output_dim, activation='relu')

    def call(self, inputs):
        # inputs shape: (Batch, Steps, Nodes, Features)
        # Tách đặc trưng thời gian (giả định nằm ở cuối) và đặc trưng không gian
        features = self.feature_embedding(inputs)
        
        # Thêm positional encoding đơn giản để mô phỏng Temporal Embedding
        # Trong thực tế, STE sẽ phức tạp hơn với Node2Vec, ở đây ta dùng Dense layers
        return features

class SpatialAttention(layers.Layer):
    """
    4.1 Spatial Attention: Tính toán tương quan giữa các nút vi và vj
    s_vi,v = <h_vi|e_vi, h_v|e_v> / sqrt(2D)
    """
    def __init__(self, n_nodes):
        super(SpatialAttention, self).__init__()
        self.W1 = layers.Dense(1)
        self.W2 = layers.Dense(1)
        self.W3 = layers.Dense(1)
        self.V = layers.Dense(n_nodes) 

    def call(self, inputs):
        # inputs: (Batch, Steps, Nodes, Features)
        # Quan tâm đến chiều Nodes
        # Shape: (Batch, Nodes, Features) (Lấy trung bình theo steps hoặc xét step cuối)
        x = tf.reduce_mean(inputs, axis=1) 
        
        # Cơ chế Attention cơ bản
        # Logic: S = V * sigmoid(W1*x + W2*x + b)
        lhs = self.W1(x) # (B, N, 1)
        rhs = self.W2(x) # (B, N, 1)
        
        # Broadcast để tạo ma trận N x N
        logits = lhs + tf.transpose(rhs, [0, 2, 1]) # (B, N, N)
        attention = tf.nn.softmax(tf.nn.sigmoid(logits), axis=-1)
        return attention # Matrix S

class TemporalAttention(layers.Layer):
    """
    4.2 Temporal Attention: Bắt giữ phụ thuộc giữa các bước thời gian
    M_tx,t
    """
    def __init__(self, n_steps):
        super(TemporalAttention, self).__init__()
        self.W1 = layers.Dense(1)
        self.W2 = layers.Dense(1)
        self.W3 = layers.Dense(1)
        self.V = layers.Dense(n_steps)

    def call(self, inputs):
        # inputs: (Batch, Steps, Nodes, Features)
        # Quan tâm đến chiều Steps, gộp Nodes
        x = tf.transpose(inputs, (0, 2, 1, 3)) # (B, N, T, F)
        x = tf.reduce_mean(x, axis=1) # (B, T, F)
        
        lhs = self.W1(x) 
        rhs = self.W2(x)
        
        logits = lhs + tf.transpose(rhs, [0, 2, 1]) # (B, T, T)
        attention = tf.nn.softmax(tf.nn.sigmoid(logits), axis=-1)
        return attention # Matrix M

class GatedFusion(layers.Layer):
    """
    4.3 Gated Fusion Module: Hợp nhất cổng
    z = sigmoid(W*XS + U*XT)
    output = z * XS + (1-z) * XT
    """
    def __init__(self, output_dim):
        super(GatedFusion, self).__init__()
        self.W = layers.Dense(output_dim)
        self.U = layers.Dense(output_dim)

    def call(self, spatial_feat, temporal_feat):
        z = tf.nn.sigmoid(self.W(spatial_feat) + self.U(temporal_feat))
        return z * spatial_feat + (1 - z) * temporal_feat

class AI_GFACN(Model):
    def __init__(self, n_nodes, input_steps, output_steps, feature_dim, hidden_dim=64):
        super(AI_GFACN, self).__init__()
        self.n_nodes = n_nodes
        self.input_steps = input_steps
        self.output_steps = output_steps
        
        # 3. STE Module
        self.ste = STE_Layer(hidden_dim, n_nodes, input_steps)
        
        # 4. Core Architecture
        # Attention
        self.spatial_attn = SpatialAttention(n_nodes)
        self.temporal_attn = TemporalAttention(input_steps)
        
        # GCN Encoder Layers (Giả lập GCN đơn giản với Dense + Adjacency)
        self.gcn_weight = layers.Dense(hidden_dim, activation='relu')
        
        # Gated Fusion
        self.fusion = GatedFusion(hidden_dim)
        
        # 5. Output FCs
        self.flatten = layers.Flatten()
        # Output shape: (Batch, Output_Steps, Nodes) -> ta reshape lại sau
        self.fc_out = layers.Dense(output_steps * n_nodes, activation=None) # Linear activation cho hồi quy

    def call(self, inputs, training=False):
        # inputs list: [A1, A2, Features]
        # A1: (Batch, Nodes, Nodes) -> (32, 25, 25)
        # A2: (Batch, Time, Nodes, Nodes) -> (32, 12, 25, 25)
        # x:  (Batch, Time, Nodes, Feats) -> (32, 12, 25, 133)
        A1, A2, x = inputs[0], inputs[1], inputs[2]
        
        # --- B1: STE Embedding ---
        x_embed = self.ste(x) # (32, 12, 25, 64)
        
        # --- B2: Attention Mechanism ---
        # 1. Spatial Attention
        S = self.spatial_attn(x_embed) # (32, 25, 25)
        S_expanded = tf.expand_dims(S, axis=1) # (32, 1, 25, 25)
        x_spatial = tf.matmul(S_expanded, x_embed, transpose_a=True) # (32, 12, 25, 64)
        
        # 2. Temporal Attention
        M = self.temporal_attn(x_embed) # (32, 12, 12)
        x_temp_trans = tf.transpose(x_embed, (0, 2, 1, 3)) # (32, 25, 12, 64)
        M_expanded = tf.expand_dims(M, axis=1) # (32, 1, 12, 12)
        x_temporal = tf.matmul(M_expanded, x_temp_trans, transpose_a=True) # (32, 25, 12, 64)
        x_temporal = tf.transpose(x_temporal, (0, 2, 1, 3)) # (32, 12, 25, 64)
        
        # --- B3: Gated Fusion ---
        x_fused = self.fusion(x_spatial, x_temporal)
        
        # --- B4: Graph Convolution (GCN) ---
        
        # SỬA LỖI TẠI ĐÂY: Hợp nhất A1 và A2
        # A1 đang là (Batch, Nodes, Nodes), cần thêm dimension Time để cộng với A2
        A1_expanded = tf.expand_dims(A1, axis=1) # (32, 1, 25, 25)
        
        # Bây giờ broadcast sẽ hoạt động: (32, 1, 25, 25) + (32, 12, 25, 25) 
        A_combined = A1_expanded + A2  # Kết quả: (32, 12, 25, 25)
        
        # Chuẩn bị cho GCN (Flatten Batch và Time)
        # Input features: (Batch * Time, Nodes, Hidden)
        batch_size = tf.shape(x_fused)[0]
        x_reshaped = tf.reshape(x_fused, (-1, self.n_nodes, x_fused.shape[-1])) # (384, 25, 64)
        
        # Input Adjacency: (Batch * Time, Nodes, Nodes)
        A_reshaped = tf.reshape(A_combined, (-1, self.n_nodes, self.n_nodes)) # (384, 25, 25)
        
        # Tính toán GCN: ReLU(A * X * W)
        # Bước 1: X * W
        x_gcn = self.gcn_weight(x_reshaped) # (384, 25, 64)
        
        # Bước 2: A * (XW)
        out_gcn = tf.matmul(A_reshaped, x_gcn) # (384, 25, 64)
        out_gcn = tf.nn.relu(out_gcn)
        
        # --- B5: Output Prediction ---
        # Flatten về (Batch, ...) để đưa vào lớp Dense cuối
        out_flat = layers.Flatten()(tf.reshape(out_gcn, (batch_size, -1)))
        
        prediction = self.fc_out(out_flat) 
        prediction = tf.reshape(prediction, (batch_size, self.output_steps, self.n_nodes))
        
        return prediction

In [16]:
import os
import json
import numpy as np
import tensorflow as tf

# --- Config ---
city = 'Seoul'
year = '2016'
n_steps_in = 12   # Input history (H=12 in description)
n_steps_out = 3   # Output prediction (J=3 in description)
batch_size = 32
epochs = 50
learning_rate = 0.001

# --- 1. Load Data (MG_TAR Pipeline) ---
print("Loading MG_TAR Data...")
# Lưu ý: n_steps trong data_loader của bạn là output steps, length là input steps
datasets = data_loader('/kaggle/input/mg-tar', city, year, length=n_steps_in, n_steps=n_steps_out, is_scale=True, temporal_copy=True)

# Lấy các biến cần thiết
risk_train, risk_test = datasets['risk'][0], datasets['risk'][1]
# ... (Load các feature khác tương tự như code mẫu của bạn: road, poi, speed, volume...)
# Để gọn, ta giả định 'node_features' đã được nối như trong code mẫu
# Tái tạo lại node_features (Feature Engineering)
def create_node_features(d, split_idx=0): # 0 for train, 1 for test
    return np.concatenate([
        d['risk'][split_idx], d['demo'][split_idx], d['poi'][split_idx], 
        d['road'][split_idx], d['volume'][split_idx], d['speed'][split_idx], 
        d['weather'][split_idx], d['calendar'][split_idx], d['dtg'][split_idx]
    ], axis=-1)

node_feat_train = create_node_features(datasets, 0)
node_feat_test = create_node_features(datasets, 1)
y_train, y_test = datasets['y'][0], datasets['y'][1] # Target

# Load Ma trận kề (Giả lập A1 và A2 từ dữ liệu có sẵn)
# AI-GFACN cần 2 ma trận: A1 (Distance), A2 (Accident Risk)
# Ta dùng A_road làm A1, A_traffic (hoặc A_risk nếu có) làm A2
import pandas as pd
A_road = pd.read_csv(f'/kaggle/input/mg-tar/graph_data/{city}-{year}-road-normalized-district-jaccard.csv', engine='c', index_col=0).to_numpy()
with open(f'/kaggle/input/mg-tar/graph_data/{city}-{year}-traffic-district-normalized-train-jaccard.npy', 'rb') as f:
    A_traffic_full = np.load(f)

# Chuẩn bị dữ liệu cho Model (Tile ma trận kề theo batch size để khớp input model)
# Lưu ý: Trong thực tế nên dùng tf.data.Dataset để tránh memory leak khi tile quá lớn
# Ở đây ta demo logic đơn giản
num_train = node_feat_train.shape[0]
num_test = node_feat_test.shape[0]
n_nodes = node_feat_train.shape[2]
n_features = node_feat_train.shape[3]

# A1: Static (Road) -> Tile
A1_train = np.tile(A_road, (num_train, 1, 1)).astype('float32')
A1_test = np.tile(A_road, (num_test, 1, 1)).astype('float32')

# A2: Dynamic (Traffic/Accident) -> Cắt từ dữ liệu traffic đã load
# Giả sử A_traffic_full tương ứng với train set
A2_train = A_traffic_full[:num_train].astype('float32')
# Với test, ta lấy phần cuối (lưu ý check size kỹ trong thực tế)
A2_test = np.tile(A_road, (num_test, 1, 1)).astype('float32') # Placeholder nếu thiếu dữ liệu test động

# --- 2. Initialize Model ---
print("Building AI-GFACN Model...")
model = AI_GFACN(n_nodes=n_nodes, 
                 input_steps=n_steps_in, 
                 output_steps=n_steps_out, 
                 feature_dim=n_features,
                 hidden_dim=64)

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.MeanSquaredError()

# --- 3. Custom Training Loop (để kiểm soát input list) ---
# Chuyển đổi sang tf.data
train_dataset = tf.data.Dataset.from_tensor_slices(((A1_train, A2_train, node_feat_train), y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices(((A1_test, A2_test, node_feat_test), y_test))
test_dataset = test_dataset.batch(batch_size)

print("Start Training...")
for epoch in range(epochs):
    # Training
    epoch_loss_avg = tf.keras.metrics.Mean()
    for inputs, targets in train_dataset: # inputs là tuple (A1, A2, Feats)
        with tf.GradientTape() as tape:
            predictions = model(inputs, training=True)
            loss = loss_fn(targets, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        epoch_loss_avg.update_state(loss)
    
    print(f"Epoch {epoch+1}/{epochs} - Loss (MSE): {epoch_loss_avg.result():.4f}")

# --- 4. Evaluation (MAE, RMSE, PCC) ---
print("\nEvaluating...")
all_preds = []
all_targets = []

for inputs, targets in test_dataset:
    preds = model(inputs, training=False)
    all_preds.append(preds.numpy())
    all_targets.append(targets.numpy())

all_preds = np.concatenate(all_preds, axis=0)
all_targets = np.concatenate(all_targets, axis=0)

# Tính toán Metrics
mae_score, rmse_score, pcc_score = metric(all_preds, all_targets)

print("="*30)
print(f"AI-GFACN Model Results on {city} {year}")
print(f"MAE : {mae_score:.4f}")
print(f"RMSE: {rmse_score:.4f}")
print(f"PCC : {pcc_score:.4f}")
print("="*30)

Loading MG_TAR Data...
Building AI-GFACN Model...
Start Training...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'ste__layer_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'spatial_attention_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'temporal_attention_3', however t

Epoch 1/50 - Loss (MSE): 3.2616
Epoch 2/50 - Loss (MSE): 2.3831
Epoch 3/50 - Loss (MSE): 2.3581
Epoch 4/50 - Loss (MSE): 2.3508
Epoch 5/50 - Loss (MSE): 2.3111
Epoch 6/50 - Loss (MSE): 2.2909
Epoch 7/50 - Loss (MSE): 2.2884
Epoch 8/50 - Loss (MSE): 2.2481
Epoch 9/50 - Loss (MSE): 2.2546
Epoch 10/50 - Loss (MSE): 2.2175
Epoch 11/50 - Loss (MSE): 2.2027
Epoch 12/50 - Loss (MSE): 2.1927
Epoch 13/50 - Loss (MSE): 2.1635
Epoch 14/50 - Loss (MSE): 2.1461
Epoch 15/50 - Loss (MSE): 2.1439
Epoch 16/50 - Loss (MSE): 2.1172
Epoch 17/50 - Loss (MSE): 2.0935
Epoch 18/50 - Loss (MSE): 2.0866
Epoch 19/50 - Loss (MSE): 2.0585
Epoch 20/50 - Loss (MSE): 2.0356
Epoch 21/50 - Loss (MSE): 2.0072
Epoch 22/50 - Loss (MSE): 1.9891
Epoch 23/50 - Loss (MSE): 1.9717
Epoch 24/50 - Loss (MSE): 1.9463
Epoch 25/50 - Loss (MSE): 1.9348
Epoch 26/50 - Loss (MSE): 1.9162
Epoch 27/50 - Loss (MSE): 1.8915
Epoch 28/50 - Loss (MSE): 1.8804
Epoch 29/50 - Loss (MSE): 1.8648
Epoch 30/50 - Loss (MSE): 1.8312
Epoch 31/50 - Loss 

InvalidArgumentError: Exception encountered when calling AI_GFACN.call().

[1m{{function_node __wrapped__BatchMatMulV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} In[0] and In[1] must have compatible batch dimensions: [1024,25,25] vs. [384,25,64] [Op:BatchMatMulV2] name: [0m

Arguments received by AI_GFACN.call():
  • inputs=('tf.Tensor(shape=(32, 25, 25), dtype=float32)', 'tf.Tensor(shape=(32, 25, 25), dtype=float32)', 'tf.Tensor(shape=(32, 12, 25, 133), dtype=float32)')
  • training=False

In [17]:
# --- SỬA LỖI DATA PREPARATION CHO A2_TEST ---

# 1. Định nghĩa lại số bước thời gian input (quan trọng)
n_steps_in = 12 

# 2. Sửa lại cách tạo A2_test (Ma trận động cho tập test)
# Lỗi cũ: A2_test = np.tile(A_road, (num_test, 1, 1)) -> Thiếu chiều Time
# Sửa mới: Tile thêm dimension n_steps_in
print("Fixing A2_test shape...")

if 'A_traffic_test' in locals() and len(A_traffic_test) == num_test:
    # Nếu bạn có dữ liệu traffic thật cho test
    A2_test = A_traffic_test.astype('float32')
else:
    # Nếu không có (fallback), lặp lại A_road 12 lần cho mỗi mẫu
    # Shape mong muốn: (num_test, 12, 25, 25)
    # A_road shape gốc: (25, 25)
    
    # Bước 1: Expand A_road thành (1, 1, 25, 25)
    A_road_expanded = np.expand_dims(np.expand_dims(A_road, axis=0), axis=0)
    
    # Bước 2: Tile thành (num_test, 12, 25, 25)
    A2_test = np.tile(A_road_expanded, (num_test, n_steps_in, 1, 1)).astype('float32')

print(f"Corrected A2_test shape: {A2_test.shape}") # Nên là (..., 12, 25, 25)

# --- TẠO LẠI DATASET VỚI A2_TEST MỚI ---
test_dataset = tf.data.Dataset.from_tensor_slices(((A1_test, A2_test, node_feat_test), y_test))
test_dataset = test_dataset.batch(batch_size)

# --- CHẠY LẠI ĐÁNH GIÁ ---
print("\nRe-evaluating...")
all_preds = []
all_targets = []

for inputs, targets in test_dataset:
    preds = model(inputs, training=False)
    all_preds.append(preds.numpy())
    all_targets.append(targets.numpy())

all_preds = np.concatenate(all_preds, axis=0)
all_targets = np.concatenate(all_targets, axis=0)

# Tính toán Metrics
mae_score, rmse_score, pcc_score = metric(all_preds, all_targets)

print("="*30)
print(f"AI-GFACN Model Results on {city} {year}")
print(f"MAE : {mae_score:.4f}")
print(f"RMSE: {rmse_score:.4f}")
print(f"PCC : {pcc_score:.4f}")
print("="*30)

Fixing A2_test shape...
Corrected A2_test shape: (141, 12, 25, 25)

Re-evaluating...
AI-GFACN Model Results on Seoul 2016
MAE : 1.1496
RMSE: 1.8325
PCC : 0.0554
